# My Rating

### 1 Prepare data
##### 1.1 Init Title Base

In [1]:
import pandas as pd
from imdb_base_data_loader import ImdbBaseDataLoader

dl = ImdbBaseDataLoader(from_cache=True)

Loading IMDb base data from cache /Users/apoltavets/anna-apps/ml-and-engineering-docs/ratings/data/cache/imdb_base.tsv.gz...
Loaded (687, 2) of my ratings


In [ ]:
from imdb_crew_data_loader import ImdbCrewDataLoader

dl_crew = ImdbCrewDataLoader(imdb_base=dl.imdb_base, from_cache=True)
tmp = dl_crew.crew_calc

In [20]:
dl.imdb_base.head(1)

,Unnamed: 0,tconst,imdb_rating,imdb_num_votes,title_type,title,start_year,end_year,runtime_minutes,genres,tconst_parent,parent_title,season,episode,num_genres,my_rating
0,0,tt33372836,8.8,156,tvEpisode,"and """"",2024,2024,23,"Animation,Comedy,Drama",tt21975478,Days with My Stepsister,1,12,3,0.0


In [ ]:
tsv = pd.read_csv("data/imdb/movies_2023_2024.tsv", sep='\t', low_memory=False, on_bad_lines='skip')

In [17]:
tmp["my_rating"] = tmp["my_rating"].fillna(0)
tmp1 = (tmp[tmp["start_year"].isin([2023, 2024])][tmp['imdb_num_votes'] > 300][tmp['imdb_num_votes'] > 300]
        .sort_values(by=['imdb_rating', 'imdb_num_votes'], ascending=[False, True])[['tconst', 'title','start_year', 'genres', 'imdb_rating', 'imdb_num_votes', 'nconst_avg_my_rating',  'nconst_avg_rating']]
    .drop_duplicates(subset=['tconst'], keep='first')
    .merge(tsv, left_on='tconst', right_on='imdb', how='inner')
)


/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_36533/2629837.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp1 = (tmp[tmp["start_year"].isin([2023, 2024])][tmp['imdb_num_votes'] > 300]


In [18]:
tmp1[tmp1['original_language'] == 'en'][tmp1['production_companies_names'].notna()]

/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_36533/2556186461.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tmp1[tmp1['original_language'] == 'en'][tmp1['production_companies_names'].notna()]


,tconst,title,start_year,genres,imdb_rating,imdb_num_votes,nconst_avg_my_rating,nconst_avg_rating,id,imdb,...,overview,popularity,release_date,vote_average,vote_count,production_companies_names,production_companies_id,production_companies_country,budget,revenue
4,tt27610971,WordLotto,2023,Comedy,9.1,354,0.0,9.100000,1275551,tt27610971,...,As 'alternative facts' began to overrun every ...,0.0792,2023-09-01,8.0,1,New Fiction,226349,NaN,50000.0,0.0
28,tt27262766,Horn and Ivory,2023,Thriller,8.7,643,0.0,8.700000,1426788,tt27262766,...,"Ambitions, dreams and hope shape one's future ...",0.1092,2023-09-15,9.0,1,Shadow Players,252000,IN,0.0,0.0
35,tt28020183,A Little Life,2023,Drama,8.6,418,0.0,8.533333,1138074,tt28020183,...,Following four college friends in New York Cit...,0.5036,2023-09-28,9.0,1,Playful Productions|Trafalgar Releasing,119326|139109,GB|,0.0,0.0
36,tt34710219,765874: Unification,2024,Sci-Fi,8.6,449,0.0,8.600000,1390127,tt34710219,...,765874: Unification is a mind-bending Star Tre...,0.2940,2024-11-18,7.0,6,OTOY,234840,US,0.0,0.0
42,tt21826846,Pollo Loco,2023,"Crime,Drama,Thriller",8.5,424,0.0,8.500000,1185348,tt21826846,...,"When a young, power-hungry drug dealer discove...",0.0797,2023-10-14,0.0,0,DD23 Productions,216253,NaN,30000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4044,tt20601506,Alien Invasion,2023,"Horror,Sci-Fi",1.9,612,0.0,1.900000,981314,tt20601506,...,"A group of friends find a large egg, but littl...",0.7130,2023-05-11,5.5,91,Proportion Productions,84624,GB,0.0,0.0
4049,tt27036391,Cocaine Shark,2023,"Crime,Drama,Horror",1.7,879,0.0,4.233333,1097322,tt27036391,...,"A mafia drug lord has unleashed a new, highly ...",0.2541,2023-09-08,3.0,23,Wild Eye Releasing|Polonia Brothers Entertainment,115963|11976,|,0.0,0.0
4050,tt19387356,Space Sharks,2024,"Comedy,Horror,Sci-Fi",1.6,340,0.0,6.366667,1143790,tt19387356,...,A military experiment conducted on a space sta...,0.9480,2024-06-23,2.1,10,SoCal Cinema Studios,136596,NaN,0.0,0.0
4051,tt10071550,Amityville: Where the Echo Lives,2024,Horror,1.5,459,0.0,1.500000,1328814,tt10071550,...,When paranormal investigator Heather West rece...,2.1736,2024-10-28,5.441,17,Ghost Box Productions,236197,NaN,0.0,0.0


In [ ]:
tmp1 = tmp1.merge(tsv, left_on='tconst', right_on='imdb', how='inner')
tmp1

In [ ]:
tmp2 = tmp1.groupby(['title', 'parent_title', 'genres']).agg({'my_rating': 'mean', 'imdb_rating': 'mean', 'imdb_num_votes': 'sum', 'nconst_avg_my_rating': 'mean', 'start_year': 'min', 'original_language': })
tmp2[(tmp2['my_rating'] == 0) & (tmp2['nconst_avg_my_rating'] > 0) & (tmp2["start_year"] > 2010)].sort_values(by=['nconst_avg_my_rating'], ascending=[False])

In [ ]:
tmp[tmp['nconst'] == 'nm0430210'].sort_values(by=['nconst_avg_my_rating'], ascending=[False])

In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from feature_engine.encoding import OrdinalEncoder
from feature_engine.scaling import MeanNormalizationScaler
X = full_df_train.drop(columns=['my_rating'])
y = full_df_train['my_rating']

categorical_cols = X.select_dtypes(include=['object']).columns.to_list()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.to_list()

transformers = [
    ArbitraryNumberImputer(arbitrary_number=0, variables=numerical_cols),
    MeanNormalizationScaler(variables=numerical_cols),
    CategoricalImputer(imputation_method='missing', variables=categorical_cols),
    OrdinalEncoder(encoding_method='ordered', variables=categorical_cols)
]

for t in transformers:
    X = t.fit_transform(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
print(f'Test score: {rf.score(X_test, y_test)}')

## Principals

In [ ]:
principals = pd.read_csv('data/imdb/title.principals.tsv.gz', sep='\t', compression='gzip', low_memory=False)
principals.merge(name, on='nconst', how='inner')
principals.head()

In [ ]:
principals[principals['tconst'] == 'tt21151974'].head(100)

In [ ]:
year = 1930
tdf = dl.imdb_base[dl.imdb_base['start_year'] == year][['tconst', 'title']].sort_values(by='tconst')
tdf.to_csv(f'data/imdb/titles_{year}.csv', index=False)
tdf.count()